## Training data Generation

12 params to run the ROI generation script:
- h5path : HDF5 file path
- Abhome : Saving directory for specific Antibody, i.e. CK5, CK8, PGP9.5
- hiclip_val : Target channel, higher limit for clipping
- lowclip_val : Target channel, lower limit for clipping
- nuc_clip_low   : Nuc channel, lower limit for clipping
- nuc_clip_high  : Nuc channel, higher limit for clipping
- cyto_clip_low  : Cyto channel, lower limit for clipping
- cyto_clip_high : Cyto channel, higher limit for clipping
- xstart : x-coordinate for selected ROI
- ystart : y-coordinate for selected ROI
- zstart : z-coordinate for selected ROI
- ROI_dim : Dimension of ROI to be cropped, 512 x 512 (default)

Return: 
* "Train" folder in specified Antibody directory, with "Ch0", "Ch1", "Ch2" as subfolder, 
* Cropped ROIs blocks are saved in this folder w/ this format: *OTLS4_NODO_mmddyy_xxxxx_Xpos_xxxxx_xxxxx_Ypos_xxxxx_xxxxx_stack_xxxxx_xxxxx_mirror* 

In [5]:
import os
import h5py as h5
from CollectImgStackFused import *
import warnings
warnings.filterwarnings("ignore")

In [6]:
def main(h5path
        ,Ab_home
        ,hiclip_val
        ,lowclip_val
        ,nuc_clip_low
        ,nuc_clip_high
        ,cyto_clip_low
        ,cyto_clip_high
        ,clahe
        ,xstart
        ,ystart
        ,zstart
        ,ROI_dim
        ,orient
        ):

    filename = h5path
    # savedir = os.getcwd() + os.sep + 'train'
    if not os.path.exists(Ab_home):
        os.mkdir(Ab_home)
    savedir = Ab_home + os.sep + 'train'
    if not os.path.exists(savedir):
        os.mkdir(savedir)
    blockname = filename.split("/")[3]
    print(blockname)
    print(savedir)

    xstop = xstart + ROI_dim
    xcoords = (xstart, xstop)

    ystop = ystart + ROI_dim
    ycoords = (ystart, ystop)

    zstop = zstart + 50
    zcoords = (zstart, zstop)

    with h5.File(filename, 'r') as f:
        collectImgStackFused(f,
                        savedir,
                        blockname,
                        zcoords,
                        xcoords,
                        ycoords,
                        orient,
                        CLAHE=clahe,
                        hiclip_val  = hiclip_val,
                        lowclip_val = lowclip_val,
                        nuc_clip_low   = nuc_clip_low,
                        nuc_clip_high  = nuc_clip_high,
                        cyto_clip_low  = cyto_clip_low,
                        cyto_clip_high = cyto_clip_high)
    f.close()

**Manual Setup**

you could preview to find the coords with bigstitcher or GUI

In [3]:
# change this
h5path = "W:/Trilabel_Data/CK8/OTLS4_NODO_6-20-23_19-068I_CK8/data-f0.h5"
Ab_home = os.getcwd()

hiclip_val = 2200
lowclip_val = 120
nuc_clip_low = 10
nuc_clip_high = 10000
cyto_clip_low = 10
cyto_clip_high = 5000
clahe = False

xstart = 200
ystart = 2500
zstart = 30
ROI_dim = 512
orient = 0 # 1 for xzy, 0 for zxy

main(h5path
    ,Ab_home
    ,hiclip_val,lowclip_val
    ,nuc_clip_low,nuc_clip_high
    ,cyto_clip_low,cyto_clip_high
    ,clahe
    ,xstart,ystart,zstart
    ,ROI_dim
    ,orient)

50
50
50


**Auto Set up with GUI** 

Using *ROI_v2.py*, export all the params into *ROI_coords_yyyy-mm-dd.csv* file

In [7]:
import pandas as pd
fname = "ROI_coords_2023-08-02.csv" #change this
df = pd.read_csv(fname)
df = df.dropna(how="all")

for i in range(len(df)):
    h5path = df["h5path"][i]
    Ab_home = df["Abhome"][i]
    hiclip_val = df["pgp_clipHigh"][i]
    lowclip_val = df["pgp_clipLow"][i]
    nuc_clip_low = df["nuc_clipLow"][i]
    nuc_clip_high = df["nuc_clipHigh"][i]
    cyto_clip_low = df["cyto_clipLow"][i]
    cyto_clip_high = df["cyto_clipHigh"][i]
    if df["pgp_ctehmt_method"][i] == "Rescale":
        clahe = False
    else:
        clahe = True    
    xstart = df["xcoord"][i]
    ystart = df["ycoord"][i]
    zstart = df["zcoord"][i]
    ROI_dim = df["ROIdim"][i]
    orient = df["orient"][i]

    main(h5path
        ,Ab_home
        ,hiclip_val,lowclip_val
        ,nuc_clip_low,nuc_clip_high
        ,cyto_clip_low,cyto_clip_high
        ,clahe
        ,xstart,ystart,zstart
        ,ROI_dim
        ,orient)

OTLS4_NODO_5-22-23_15-071K_PGP9.5
Y:/Sarah/Test PGP\PGP9.5\train
Y:/Sarah/Test PGP\PGP9.5\train\ch1\OTLS4_NODO_5-22-23_15-071K_PGP9.5_Xpos_000863_001563_Ypos_001944_002644_stack_000016_000066
reading img s01
Y:/Sarah/Test PGP\PGP9.5\train\ch0\OTLS4_NODO_5-22-23_15-071K_PGP9.5_Xpos_000863_001563_Ypos_001944_002644_stack_000016_000066
reading img s02
Y:/Sarah/Test PGP\PGP9.5\train\ch2\OTLS4_NODO_5-22-23_15-071K_PGP9.5_Xpos_000863_001563_Ypos_001944_002644_stack_000016_000066
reading img s00
OTLS4_NODO_6-11-23_16-010J_PGP9.5
Y:/Sarah/Test PGP\PGP9.5\train
Y:/Sarah/Test PGP\PGP9.5\train\ch1\OTLS4_NODO_6-11-23_16-010J_PGP9.5_Xpos_000061_000761_Ypos_002792_003492_stack_000044_000094
reading img s01
Y:/Sarah/Test PGP\PGP9.5\train\ch0\OTLS4_NODO_6-11-23_16-010J_PGP9.5_Xpos_000061_000761_Ypos_002792_003492_stack_000044_000094
reading img s02
Y:/Sarah/Test PGP\PGP9.5\train\ch2\OTLS4_NODO_6-11-23_16-010J_PGP9.5_Xpos_000061_000761_Ypos_002792_003492_stack_000044_000094
reading img s00
